# Py City School District Analysis

In [1]:
import pandas as pd
school_data = pd.read_csv('Resources/schools_complete.csv')
student_data = pd.read_csv('Resources/students_complete.csv')

complete_school_data = pd.merge(student_data, school_data, how="left", on="school_name")
# complete_school_data.head() - verified merge

## Creating District Summary Table

In [2]:
#Calculating number of schools:
schools = len(school_data.school_name)

#Calulating number of students and storing a formatted version for the table:
students = student_data.student_name.count()
students_formatted = '{:,.0f}'.format(students)

#Calulating budget and storing a formatted version for the table:
budget = school_data.budget.sum()
budget_formatted = '${:,.2f}'.format(float(budget))

#Calculating average math and reading scores:
avg_math = round(student_data.math_score.mean(), 6)
avg_reading = round(student_data.reading_score.mean(),6)

#Verified outputs
#print(schools, students, students_formatted, budget, budget_formatted, avg_math, avg_reading)

#Calculating passing percentages:
pct_pass_math = (len(student_data.loc[(student_data['math_score']>=70)])/students)*100
pct_pass_reading = (len(student_data.loc[(student_data['reading_score']>=70)])/students)*100
pct_pass_combined = (len(student_data.loc[(student_data['math_score']>=70) & 
                                                (student_data['reading_score']>=70)])/students)*100
#Verified outputs
#print(pct_pass_math, pct_pass_reading, pct_pass_combined)

#Building Data Frame
district_summary = pd.DataFrame({'Total Schools':[schools],
                                 'Total Students': [students_formatted],
                                 'Total Budget': [budget_formatted],
                                 'Average Math Score': [avg_math],
                                 'Average Reading Score': [avg_reading],
                                 '% Passing Math': [pct_pass_math],
                                 '% Passing Reading': [pct_pass_reading],
                                 '% Overall Passing': [pct_pass_combined]
                                })

district_summary.round(2)

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98,85.81,65.17


## Creating School Summary Table

In [3]:
# Using school_data as starting point for school summary table
school_summary = school_data.copy()
school_summary.drop('School ID', axis=1, inplace=True)
school_summary.set_index('school_name', inplace=True)
school_summary.sort_index(inplace=True)
# school_summary verified output

# Calculating Budget per Student and adding column
school_summary['Budget per Student'] = school_summary.budget/school_summary['size']
# school_summary - verified output

#Calculating Average Math and Reading Scores and adding concatenating the two DFs
avg_scores = complete_school_data.groupby('school_name').agg({'math_score': ['mean'],
                                                            'reading_score': ['mean'],
                                                            })
school_summary = pd.concat([school_summary, avg_scores], axis=1)
# school_summary - verified output

# Calculating Percent Passing Math and adding column
math_groupby = complete_school_data.loc[complete_school_data.math_score >= 70, :].groupby("school_name").count()
pct_math_passing = (math_groupby.math_score/school_summary['size'])*100
school_summary['% Passing Math'] = pct_math_passing

# Calculating Percent Passing Reading and adding column
reading_groupby = complete_school_data.loc[complete_school_data.reading_score >= 70, :].groupby("school_name").count()
pct_reading_passing = (reading_groupby.reading_score/school_summary['size'])*100
school_summary['% Passing Reading'] = pct_reading_passing

# Calculating Percent Overall Passing and adding column
overall_groupby = complete_school_data.loc[(complete_school_data.math_score >= 70) &
                                            (complete_school_data.reading_score >= 70),:].groupby("school_name").count()
pct_overall_passing = (overall_groupby.grade/school_summary['size'])*100
school_summary['% Overall Passing'] = pct_overall_passing
# school_summary - verified output

# Confirming data types prior to formatting
# school_summary.info()

#storing the unformatted variable for use later
budget_per_student = school_summary['Budget per Student']

# Formatting and renaming columns of School Summary Table
school_summary['size'] = school_summary['size'].map('{:,.0f}'.format)
school_summary['budget'] = school_summary['budget'].map('${:,.2f}'.format)
school_summary['Budget per Student'] = school_summary['Budget per Student'].map('${:,.2f}'.format)
school_summary.columns = ['Type', 'Total Students', 'Total Budget', 'Budget per Student',
                          'Average Math Score', 'Average Reading Score', '% Passing Math',
                         '% Passing Reading', '% Overall Passing'
                         ]
school_summary.round(2)

,Type,Total Students,Total Budget,Budget per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05,81.03,66.68,81.93,54.64
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,91.33
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,53.20
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,68.31,79.30,54.29
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39,97.14,90.60
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75,80.86,53.53
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51,96.25,89.23
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,53.51
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,53.54


## Top Performing Schools (By % Overall Passing)

In [4]:
# Sort and display the top five performing schools by % overall passing.
top_performing_schools = school_summary.sort_values('% Overall Passing', ascending=False)
top_performing_schools.drop(top_performing_schools.tail(10).index, inplace = True)

top_performing_schools.round(2)

,Type,Total Students,Total Budget,Budget per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,91.33
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42,83.85,93.27,97.31,90.95
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39,97.14,90.60
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27,83.99,93.87,96.54,90.58
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59,95.95,90.54


## Bottom Performing Schools (By % Overall Passing)

In [5]:
# Sort and display the five worst-performing schools by % overall passing.
bottom_perfoming_schools = school_summary.sort_values('% Overall Passing')
bottom_perfoming_schools.drop(bottom_perfoming_schools.tail(10).index, inplace = True)

bottom_perfoming_schools.round(2)

,Type,Total Students,Total Budget,Budget per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84,80.74,66.37,80.22,52.99
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,53.20
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,53.51
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75,80.86,53.53
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,53.54


## Math Scores by Grade

In [6]:
# Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.
math_scores_by_grade = complete_school_data.groupby(['school_name', 'grade'])['math_score'].mean()

# Using .unstack to move grade index to columns and create dataframe
math_grade = math_scores_by_grade.unstack(level=-1)

#Rearranging columns to improve readability
math_grade = math_grade[['9th', '10th', '11th', '12th']]

math_grade.round(2)

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Scores by Grade

In [7]:
# Create a table that lists the average Reading Score for students of each grade level 
#(9th, 10th, 11th, 12th) at each school.
reading_scores_by_grade = complete_school_data.groupby(['school_name', 'grade'])['reading_score'].mean()

# Using .unstack to move grade index to columns and create dataframe
reading_grade = reading_scores_by_grade.unstack(level=-1)

#Rearranging columns to improve readability
reading_grade = reading_grade[['9th', '10th', '11th', '12th']]

reading_grade.round(2)

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

In [8]:
# I opted to rebuild this table from scratch instead of using the school summary table. 
# While it would be faster to group the school summary table by school spending per student and find the average of
# each column, the resulting numbers would contain discrepencies as the individual averages were created with different
# numbers of students.

school_spending = complete_school_data[['size','budget','math_score','reading_score']].copy()
school_spending['Spending per Student'] = school_spending['budget']/school_spending['size']

# school_spending.head()

# Determining bin sizes for approx 4 schools per bin
# budget_per_student = budget_per_student.sort_values()
# print(budget_per_student[0], budget_per_student[3], budget_per_student[7], budget_per_student[11], budget_per_student[14])

# Creating bins, and grouping data
bins = [0, 584, 630, 645, 680]
labels = ['<$585', '$585-630', '$630-645', '$645-680']

school_spending['Spending Ranges per Student'] = pd.cut(school_spending['Spending per Student'], bins=bins, labels=labels)
school_spending_grouped = school_spending.groupby('Spending Ranges per Student').agg({'size': 'count',
                                                                                     'math_score': 'mean',
                                                                                     'reading_score': 'mean'
                                                                                    })


# Calculating Percent Passing and adding columns
math_groupby = school_spending.loc[(school_spending.math_score >= 70),:].groupby('Spending Ranges per Student').count()
pct_math_passing = (math_groupby['size']/school_spending_grouped['size'])*100
school_spending_grouped['% Passing Math'] = pct_math_passing

reading_groupby = school_spending.loc[(school_spending.reading_score >= 70),:].groupby('Spending Ranges per Student').count()
pct_reading_passing = (reading_groupby['size']/school_spending_grouped['size'])*100
school_spending_grouped['% Passing Reading'] = pct_reading_passing

overall_groupby = school_spending.loc[(school_spending.math_score >= 70) &
                                    (school_spending.reading_score >= 70),:].groupby('Spending Ranges per Student').count()
pct_overall_passing = (overall_groupby['size']/school_spending_grouped['size'])*100
school_spending_grouped['% Overall Passing'] = pct_overall_passing

school_spending_grouped.drop('size', axis=1, inplace=True)

school_spending_grouped.columns = ['Average Math Score', 'Average Reading Score',
                                  '% Passing Math', '% Passing Reading', '% Overall Passing'
                                  ]

school_spending_grouped.round(2)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges per Student,,,,,
<$585,83.36,83.96,93.70,96.69,90.64
$585-630,79.98,82.31,79.11,88.51,70.94
$630-645,77.82,81.30,70.62,82.60,58.84
$645-680,77.05,81.01,66.23,81.11,53.53


## Scores by School Size

In [9]:
# I opted to rebuild this table from scratch instead of using the school summary table. 
# While it would be faster to group the school summary table by school size and find the average of
# each column, the resulting numbers would contain discrepencies as the individual averages were created 
# with different numbers of students.

school_size = complete_school_data[['size','math_score','reading_score']].copy()

sizes = school_data['size'].sort_values()
sizes = sizes.reset_index(drop=True)
#sizes
#print(sizes[0], sizes[4], sizes[9], sizes[14])

# Creating bins, and grouping data
bins = [400, 1999, 4000, 5000]
labels = ['Small(<2000)', 'Medium(2,000-4,000)', 'Large(4,000-5,000)']

school_size['Size Range'] = pd.cut(school_size['size'], bins=bins, labels=labels)
school_size_grouped = school_size.groupby('Size Range').agg({'size': 'count',
                                                            'math_score': 'mean',
                                                            'reading_score': 'mean'
                                                            })
# Calculating Percent Passing and adding columns
math_groupby = school_size.loc[(school_size.math_score >= 70)].groupby('Size Range').count()
pct_math_passing = (math_groupby['size']/school_size_grouped['size'])*100
school_size_grouped['% Passing Math'] = pct_math_passing

reading_groupby = school_size.loc[(school_size.reading_score >= 70),:].groupby('Size Range').count()
pct_reading_passing = (reading_groupby['size']/school_size_grouped['size'])*100
school_size_grouped['% Passing Reading'] = pct_reading_passing

overall_groupby = school_size.loc[(school_size.math_score >= 70) &
                                    (school_size.reading_score >= 70),:].groupby('Size Range').count()
pct_overall_passing = (overall_groupby['size']/school_size_grouped['size'])*100
school_size_grouped['% Overall Passing'] = pct_overall_passing

school_size_grouped.drop('size', axis=1, inplace=True)

school_size_grouped.columns = ['Average Math Score', 'Average Reading Score',
                                  '% Passing Math', '% Passing Reading', '% Overall Passing'
                                  ]

school_size_grouped.round(2)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Size Range,,,,,
Small(<2000),83.44,83.88,93.66,96.67,90.56
"Medium(2,000-4,000)",77.81,81.41,70.73,82.87,59.14
"Large(4,000-5,000)",77.13,80.98,66.50,81.35,53.92


## Scores by School Type

In [10]:
# I opted to rebuild this table from scratch instead of using the school summary table. 
# While it would be faster to group the school summary table by school type and find the average of
# each column, the resulting numbers would contain discrepencies as the individual averages were created 
# with different numbers of students.

school_type = complete_school_data[['type','size','math_score','reading_score']].copy()

school_type_grouped = school_type.groupby(school_type['type']).agg({'size': 'count',
                                                            'math_score': 'mean',
                                                            'reading_score': 'mean'
                                                            })

# school_type_grouped

# Calculating Percent Passing and adding columns
math_groupby = school_type.loc[(school_type.math_score >= 70)].groupby('type').count()
pct_math_passing = (math_groupby['size']/school_type_grouped['size'])*100
school_type_grouped['% Passing Math'] = pct_math_passing

reading_groupby = school_type.loc[(school_type.reading_score >= 70),:].groupby('type').count()
pct_reading_passing = (reading_groupby['size']/school_type_grouped['size'])*100
school_type_grouped['% Passing Reading'] = pct_reading_passing

overall_groupby = school_type.loc[(school_type.math_score >= 70) &
                                        (school_type.reading_score >= 70),:].groupby('type').count()
pct_overall_passing = (overall_groupby['size']/school_type_grouped['size'])*100
school_type_grouped['% Overall Passing'] = pct_overall_passing

school_type_grouped.drop('size', axis=1, inplace=True)

school_type_grouped.columns = ['Average Math Score', 'Average Reading Score',
                                  '% Passing Math', '% Passing Reading', '% Overall Passing'
                                  ]

school_type_grouped

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
type,,,,,
Charter,83.406183,83.902821,93.701821,96.645891,90.560932
District,76.987026,80.962485,66.518387,80.905249,53.695878
